In [1]:
cd /home/ubuntu/DistMEC/

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from classes.User import *
from classes.solver import *
from classes.Server import *
from classes.distributed_utils import *

from tqdm import tqdm, tqdm_notebook
from time import sleep
import pickle
import os
import numpy as np

In [3]:
regret_worst_nc_dict_list = []
coll_worst_nc_dict_list = []

regret_rsv_dict_list = []
regret_worst_dict_list = []
regret_central_dict_list = []
regret_cstat_dict_list = []
coll_rsv_dict_list = []
coll_worst_dict_list = []

# System Parameters
T = 10000
num_users = 4
num_svrs_list = [5,10,15,20]
num_locs = 9
# mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
ceiling = 40
# w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
space_1d_dim = 10 #4 for small, 10 for big
svr_offset_uniform = 2
max_dist = 8

num_explore = 1
trials = 5

# result dictionaries
result_dicts = []
performance_list = []
user_dicts = []

save_name = 'results/22_06_16_ceiling_20/'
isExist = os.path.exists(save_name)
if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(save_name)


# Generate visitng locations and server locations
usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs,offset = 1.7)
# svr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_svrs,offset = 1.7)

with open('traces_P_40k.pkl', 'rb') as handle:
    P_collection = pickle.load(handle)
    
with open('traces_P_40k_dict.pkl', 'rb') as handle:
    P_dict = pickle.load(handle)

In [4]:
for zt in range(len(num_svrs_list)):
    # Recording Mechanism
    regret_centralized_dict = {}

    regret_worst_dict = {}
    collisions_worst_dict = {}

    regret_rsv_dict = {}
    collisions_rsv_dict = {}

    regret_cent_stat = {}
    
    regret_worst_nc_dict = {}
    collisions_worst_nc_dict = {}
    
    num_svrs = num_svrs_list[zt]
    mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])


    for tri in range(trials):

        # Build artificial P --> 0.95 Self transition, 0.05/S-1 other transitions
        P = np.zeros(P_collection.shape)

        temp_pm = 0.05

        for i in range(P.shape[0]):
            for j in range(P.shape[0]):
                if i != j:
                    P[i,j] = temp_pm/(P.shape[0]-1)
                else:
                    P[i,j] = 1-temp_pm

        svr_locs = gen_rand_locs(space_1d=space_1d_dim - svr_offset_uniform, nums=num_svrs)
#         svr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_svrs,offset = 1.7)
        # Create Users
        Users_w = []
        for i in range(num_users):
#             P_temp = P_collection
            Users_w += [User(usr_locs,svr_locs,mu[i],i,
                          max_dist = max_dist, threshold_dist = 6, self_weight = 0.95,
                             P = P, ceiling = 1, sticky_mode = True, kick_mode=True)]
    
        Users_w_nc = []
        for i in range(num_users):
            Users_w_nc += [User(usr_locs,svr_locs,mu[i],i,
                          max_dist = 1000000, threshold_dist = 6, self_weight = 0.95,
                             P = P, ceiling = 1, sticky_mode = True, kick_mode=True)]
        

        # Create Servers
        Servers = []
        for i in range(num_svrs):
            Servers += [Server(svr_locs[i],mu,i)]

        # Recorded Values - reservation
        regret_w = np.zeros(T) # Worst
        collision_count_w = np.zeros(T)
        optimal_reward = np.zeros(T)
        
        regret_w_nc = np.zeros(T)
        collision_count_w_nc = np.zeros(T)

        # Explore rounds are common/shared across all users
        explore_rounds(Users_w, num_users, Servers, mu, regret_w, collision_count_w, optimal_reward,
                       usr_move_flag = True, rounds=num_explore)
        
        explore_rounds(Users_w_nc, num_users, Servers, mu, regret_w_nc, collision_count_w_nc, optimal_reward,
                       usr_move_flag = True, rounds=num_explore)

        round_start = ((num_svrs)*num_explore)+1


        for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):
            w = obtain_w(Users_w, len(Users_w), len(Servers))
            optimal = offline_optimal_action(w, mu)
            optimal_reward[t] = optimal[1]

            # Distributed solution 
            play_round(Users_w, Servers, mu, regret_w, collision_count_w, 
                       usr_move_flag = True, debugger = False, reservation_mode = True, optimal =optimal)
            play_round(Users_w_nc, Servers, mu, regret_w_nc, collision_count_w_nc, 
                       usr_move_flag = False, debugger = False, reservation_mode = True, optimal =optimal, w = w)
            copy_usr_loc(Users_w, Users_w_nc)

        regret_worst_dict[tri] = copy.deepcopy(regret_w)
        collisions_worst_dict[tri] = copy.deepcopy(collision_count_w)

        regret_worst_nc_dict[tri] = copy.deepcopy(regret_w_nc)
        collisions_worst_nc_dict[tri] = copy.deepcopy(collision_count_w_nc)


        del Users_w, Users_w_nc, Servers
    regret_worst_dict_list += [regret_worst_dict]
    coll_worst_dict_list += [collisions_worst_dict]
    regret_worst_nc_dict_list += [regret_worst_nc_dict]
    coll_worst_nc_dict_list += [collisions_worst_nc_dict]

<ipython-input-4-7fd647755459>:74: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):


In [5]:
save_dict = {}
save_dict['regret_worst_dict_list'] = regret_worst_dict_list
save_dict['coll_worst_dict_list'] = coll_worst_dict_list

save_dict['regret_worst_nc_dict_list'] = regret_worst_nc_dict_list
save_dict['coll_worst_nc_dict_list'] = coll_worst_nc_dict_list


with open("experiments/save/exp6/220725_greedyhidist_run1_t5.pkl", "wb") as tf:
    pickle.dump(save_dict,tf)

In [6]:
with open("experiments/save/exp6/220725_greedyhidist_run1_t5.pkl", "rb") as handle:
    load_dict = pickle.load(handle)
    
regret_worst_dict_list = load_dict['regret_worst_dict_list']
coll_worst_dict_list = load_dict['coll_worst_dict_list']

regret_worst_nc_dict_list = load_dict['regret_worst_nc_dict_list']
coll_worst_nc_dict_list = load_dict['coll_worst_nc_dict_list']